# Lecture 1 - Week 1

In [8]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
from memory_profiler import memory_usage

In [9]:
%load_ext rpy2.ipython
%load_ext memory_profiler

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


####

## 1) Here we are getting the data what we want using the figshare API

Defining the endpoint and Header info

Read more on figshare API here 
https://docs.figshare.com

In [3]:
%pwd

'/Users/ggeorg02/Desktop/DSCI 525 lectures/Week1'

In [3]:
%cd /Users/ggeorg02/Desktop/

/Users/ggeorg02/Desktop


In [13]:
# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "rainfall/"

Here we are sending a GET request to list the available files

In [14]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'is_link_only': False,
  'name': 'daily_rainfall_2014.png',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'id': 26579150,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'size': 58863},
 {'is_link_only': False,
  'name': 'environment.yml',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'id': 26579171,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'size': 192},
 {'is_link_only': False,
  'name': 'README.md',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'id': 26586554,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'size': 5422},
 {'is_link_only': False,
  'name': 'data.zip',
  'supplied_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'computed_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'id': 26766812,
  'download_url': 'https://

We are going to get the file named individual_years.zip

In [15]:
%%time
files_to_dl = ["data.zip"]  # feel free to add other files here
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

Wall time: 1min 59s


In [16]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

Wall time: 41.6 s


####

## 2) Combine the data

In [19]:
### just listing to get an idea how individual file looks like 
#use_cols = ["ArrDelay", "DepDelay", "Distance", "TailNum","UniqueCarrier","Origin","Dest"]
df = pd.read_csv("rainfall/ACCESS-CM2_daily_rainfall_NSW.csv" ,dtype={'TailNum': 'str'})
df

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
0,1889-01-01 12:00:00,-36.25,-35.00,140.625,142.50,3.293256e-13
1,1889-01-02 12:00:00,-36.25,-35.00,140.625,142.50,0.000000e+00
2,1889-01-03 12:00:00,-36.25,-35.00,140.625,142.50,0.000000e+00
3,1889-01-04 12:00:00,-36.25,-35.00,140.625,142.50,0.000000e+00
4,1889-01-05 12:00:00,-36.25,-35.00,140.625,142.50,1.047658e-02
...,...,...,...,...,...,...
1932835,2014-12-27 12:00:00,-30.00,-28.75,151.875,153.75,2.951144e-02
1932836,2014-12-28 12:00:00,-30.00,-28.75,151.875,153.75,2.257118e-01
1932837,2014-12-29 12:00:00,-30.00,-28.75,151.875,153.75,1.204670e-01
1932838,2014-12-30 12:00:00,-30.00,-28.75,151.875,153.75,2.632404e-02


In [17]:
%%time
%memit
# Shows time that regular python takes to merge file
# Join all data together
## here we are using a normal python way of merging the data 
import pandas as pd
#use_cols = ["ArrDelay", "DepDelay", "Distance", "TailNum","UniqueCarrier","Origin","Dest"]
files = glob.glob('rainfall/*.csv')
df = pd.concat((pd.read_csv(file, index_col=0)
                .assign(model=re.findall(r'[^\\]+(?=\_d)', file)[0])
                for file in files)
              )
df.to_csv("rainfall/combined_data.csv")

peak memory: 2097.82 MiB, increment: 0.59 MiB
Wall time: 11min 6s


In [12]:
files

['figshareairline\\ACCESS-CM2_daily_rainfall_NSW.csv',
 'figshareairline\\ACCESS-ESM1-5_daily_rainfall_NSW.csv',
 'figshareairline\\AWI-ESM-1-1-LR_daily_rainfall_NSW.csv',
 'figshareairline\\BCC-CSM2-MR_daily_rainfall_NSW.csv',
 'figshareairline\\BCC-ESM1_daily_rainfall_NSW.csv',
 'figshareairline\\CanESM5_daily_rainfall_NSW.csv',
 'figshareairline\\CMCC-CM2-HR4_daily_rainfall_NSW.csv',
 'figshareairline\\CMCC-CM2-SR5_daily_rainfall_NSW.csv',
 'figshareairline\\CMCC-ESM2_daily_rainfall_NSW.csv',
 'figshareairline\\EC-Earth3-Veg-LR_daily_rainfall_NSW.csv',
 'figshareairline\\FGOALS-f3-L_daily_rainfall_NSW.csv',
 'figshareairline\\FGOALS-g3_daily_rainfall_NSW.csv',
 'figshareairline\\GFDL-CM4_daily_rainfall_NSW.csv',
 'figshareairline\\GFDL-ESM4_daily_rainfall_NSW.csv',
 'figshareairline\\INM-CM4-8_daily_rainfall_NSW.csv',
 'figshareairline\\INM-CM5-0_daily_rainfall_NSW.csv',
 'figshareairline\\KIOST-ESM_daily_rainfall_NSW.csv',
 'figshareairline\\MIROC6_daily_rainfall_NSW.csv',
 'figsha

In [18]:
data1 = pd.read_csv('rainfall/combined_data.csv')
data1.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-36.25,-35.0,140.625,142.5,3.293256e-13,ACCESS-CM2
1,1889-01-02 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
2,1889-01-03 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
3,1889-01-04 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
4,1889-01-05 12:00:00,-36.25,-35.0,140.625,142.5,1.047658e-02,ACCESS-CM2


In [4]:
%%sh
du -sh figshareairline/combined_data.csv

3.1G	figshareairline/combined_data.csv


In [13]:
%%time
df = pd.read_csv("rainfall/combined_data.csv")

CPU times: user 35.8 s, sys: 9.7 s, total: 45.5 s
Wall time: 57.1 s


In [14]:
print(df.shape)

(86289323, 8)


In [15]:
df.head()

,UniqueCarrier,TailNum,ArrDelay,DepDelay,Origin,Dest,Distance,year
0,DL,N673DL,66.0,69.0,ATL,PHX,1587.0,1996
1,DL,N686DA,3.0,1.0,ATL,PHX,1587.0,1996
2,DL,N685DA,52.0,26.0,ATL,PHX,1587.0,1996
3,DL,N522DA,84.0,100.0,ATL,PHX,1587.0,1996
4,DL,N2824W,56.0,84.0,ATL,PHX,1587.0,1996


##

# Lecture 2 - Week 1

In [5]:
import dask.dataframe as dd
import pandas as pd

## Dask 

This is a scalable python library , please check it out 
https://dask.org    

It does the chunking and parallel execution for us. So we dont have to manually take care of it using the chunk_size https://cmdlinetips.com/2018/01/how-to-load-a-massive-file-as-small-chunks-in-pandas/ for chunking up ( we are using chunk_size later in this notebook to load the combined big csv to memory as chunks) or the "multiprocessing" python library for parallel processing https://docs.python.org/3/library/multiprocessing.html. 

Best part of dask is the syntax is much similar to the default pandas

In [17]:
%%time
%%memit
# shows time that dask take to merge
use_cols = ["ArrDelay", "DepDelay", "Distance", "TailNum","UniqueCarrier","Origin","Dest"]
# ddf = dd.read_csv("/Users/ggeorg02/Desktop/figshareairline/individual_years/*.csv",assume_missing=True,encoding = "ISO-8859-1",dtype={'TailNum': 'str'},usecols=use_cols)
ddf = dd.read_csv("figshareairline/individual_years/*.csv",assume_missing=True,usecols=use_cols)
ddf.to_csv("figshareairline/combined_data_dask.csv", single_file=True)

peak memory: 33366.67 MiB, increment: 18188.23 MiB
CPU times: user 5min 52s, sys: 11.5 s, total: 6min 3s
Wall time: 5min 10s


In [20]:
ddf.head()

,UniqueCarrier,TailNum,ArrDelay,DepDelay,Origin,Dest,Distance
0,UA,N7298U,15.0,12.0,ORD,PHL,678.0
1,UA,N7449U,1.0,3.0,ORD,PHL,678.0
2,UA,N7453U,-5.0,4.0,ORD,PHL,678.0
3,UA,N7288U,-9.0,0.0,ORD,PHL,678.0
4,UA,N7275U,-6.0,0.0,ORD,PHL,678.0


##

## 3) Loading the combined csv file and doing something (various ways)

In [8]:
%%time
%%memit
#simple pandas - This is how we do normally ,which means we are loading the entire data to the memory
df = pd.read_csv("figshareairline/combined_data.csv")
print(df["UniqueCarrier"].value_counts())

WN    13194660
DL    10435886
AA     9672922
UA     8821384
US     8286980
NW     6946627
CO     4976761
MQ     3954895
OO     3090853
XE     2350309
HP     2224941
AS     2162672
TW     1890420
EV     1697172
OH     1464176
FL     1265138
YV      854056
B6      811341
DH      693047
9E      521059
F9      336958
HA      274265
TZ      208420
AQ      154381
Name: UniqueCarrier, dtype: int64
peak memory: 33909.09 MiB, increment: 18972.36 MiB
CPU times: user 40.7 s, sys: 9.91 s, total: 50.6 s
Wall time: 58.9 s


In [22]:
df.head()

,UniqueCarrier,TailNum,ArrDelay,DepDelay,Origin,Dest,Distance,year
0,DL,N673DL,66.0,69.0,ATL,PHX,1587.0,1996
1,DL,N686DA,3.0,1.0,ATL,PHX,1587.0,1996
2,DL,N685DA,52.0,26.0,ATL,PHX,1587.0,1996
3,DL,N522DA,84.0,100.0,ATL,PHX,1587.0,1996
4,DL,N2824W,56.0,84.0,ATL,PHX,1587.0,1996


In [23]:
df.dtypes

UniqueCarrier     object
TailNum           object
ArrDelay         float64
DepDelay         float64
Origin            object
Dest              object
Distance         float64
year               int64
dtype: object

In [24]:
print(f"Memory usage with float64: {df[['ArrDelay','DepDelay','Distance']].memory_usage().sum() / 1e6:.2f} MB")
print(f"Memory usage with float32: {df[['ArrDelay','DepDelay','Distance']].astype('float32', errors='ignore').memory_usage().sum() / 1e6:.2f} MB")

Memory usage with float64: 2070.94 MB
Memory usage with float32: 1035.47 MB


In the next cell we are seeing an example of how to load data in small chunks. We can pass chunksize as a parameter while using read_csv. Read more on it here 
https://cmdlinetips.com/2018/01/how-to-load-a-massive-file-as-small-chunks-in-pandas/

In [9]:
%%time
%%memit
counts = pd.Series(dtype=int)
for chunk in pd.read_csv("figshareairline/combined_data.csv", chunksize=10_000_000):
    counts = counts.add(chunk["UniqueCarrier"].value_counts(), fill_value=0)
print(counts.astype(int))

9E      521059
AA     9672922
AQ      154381
AS     2162672
B6      811341
CO     4976761
DH      693047
DL    10435886
EV     1697172
F9      336958
FL     1265138
HA      274265
HP     2224941
MQ     3954895
NW     6946627
OH     1464176
OO     3090853
TW     1890420
TZ      208420
UA     8821384
US     8286980
WN    13194660
XE     2350309
YV      854056
dtype: int64
peak memory: 21020.25 MiB, increment: 6865.48 MiB
CPU times: user 39.3 s, sys: 2.75 s, total: 42 s
Wall time: 43.1 s


In [10]:
%%time
%%memit
# dask way
# Here again I am using dask to read that csv file. Remember internally its loading chunks and doing it parallely.
# here see cpu time greater than wall time 
ddf = dd.read_csv('figshareairline/combined_data.csv')
print(ddf["UniqueCarrier"].value_counts().compute())

WN    13194660
DL    10435886
AA     9672922
UA     8821384
US     8286980
NW     6946627
CO     4976761
MQ     3954895
OO     3090853
XE     2350309
HP     2224941
AS     2162672
TW     1890420
EV     1697172
OH     1464176
FL     1265138
YV      854056
B6      811341
DH      693047
9E      521059
F9      336958
HA      274265
TZ      208420
AQ      154381
Name: UniqueCarrier, dtype: int64
peak memory: 23640.69 MiB, increment: 5471.92 MiB
CPU times: user 46.4 s, sys: 6.74 s, total: 53.1 s
Wall time: 29.2 s


##

## 4) Advanced file formats

#### Arrow file format

In [4]:
import pandas as pd
## install the packages https://arrow.apache.org/docs/python/install.html
import pyarrow.dataset as ds
import pyarrow as pa
import pyarrow.parquet as pq
## How to install put instructions https://anaconda.org/conda-forge/rpy2
import rpy2.rinterface
# install this https://pypi.org/project/rpy2-arrow/#description  pip install rpy2-arrow
# have to install this as well conda install -c conda-forge r-arrow 
import rpy2_arrow.pyarrow_rarrow as pyra
### instruction
import pyarrow.feather as feather

/Users/ggeorg02/opt/miniconda3/envs/525/lib/python3.9/site-packages/rpy2_arrow/pyarrow_rarrow.py:16: UserWarning: This was designed againt arrow versions starting with 2.0. but you have 3.0.0
  warnings.warn(


In [5]:
%%R
#just seeing if its available
library("arrow")
library("dplyr")

R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [6]:
%%time
%%memit
## read more on the datasets here  https://arrow.apache.org/docs/python/dataset.html
dataset = ds.dataset("figshareairline/combined_data.csv", format="csv")
## this is of arrow table format
table = dataset.to_table()

peak memory: 16988.36 MiB, increment: 15724.98 MiB
CPU times: user 14.3 s, sys: 4.38 s, total: 18.6 s
Wall time: 18.4 s


In [30]:
%%time
# experiment in writing in feather format 
feather.write_feather(table, 'figshareairline/example.feather')

CPU times: user 5.33 s, sys: 7.03 s, total: 12.4 s
Wall time: 5.15 s


In [31]:
#### parquet file format

In [32]:
%%time
## writing as a single parquet 
pq.write_table(table, 'figshareairline/example.parquet')

CPU times: user 15.3 s, sys: 530 ms, total: 15.8 s
Wall time: 15.9 s


In [33]:
%%time
## writing as a partitioned parquet 
pq.write_to_dataset(table, 'figshareairline/example_partitioned.parquet',partition_cols=['year'])

CPU times: user 37.9 s, sys: 16.1 s, total: 54 s
Wall time: 1min 3s


## Now we have written the data in 3 different formats 
- csv
- feather 
- parquet

In [7]:
%%sh
# I am just seeing the size of the csv data
du -sh figshareairline/combined_data.csv
# I am just seeing the size of the feather data
du -sh figshareairline/example.feather
# I am just seeing the size of the parquet data
du -sh figshareairline/example.parquet
# I am just seeing the size of the parquet data - partitioned
du -sh figshareairline/example_partitioned.parquet

3.1G	figshareairline/combined_data.csv
2.1G	figshareairline/example.feather
416M	figshareairline/example.parquet
397M	figshareairline/example_partitioned.parquet


####

## 5) Experimenting the exchange of data b/w python and R.

In [ ]:
## first I am trying the normal way that anyone would

In [22]:
%%time
%%memit
#simple pandas - This is how we do normally ,which means we are loading the entire data to the memory
# just loading 10 million rows
df = pd.read_csv("figshareairline/combined_data.csv",nrows = 10_000_000)

peak memory: 15950.16 MiB, increment: 0.03 MiB
CPU times: user 4.45 s, sys: 2.38 s, total: 6.83 s
Wall time: 8.55 s


In [17]:
%%time
%%R -i df
## here I am transferring the python dataframe to R
## Look how much time it takes ! or tell me how much time you end up waiting !!
# if you see here this operation takes less time inside the R terminal, but overall it takes time , 
## The rest of time in spent on serialization and deserialization
start_time <- Sys.time()
library(dplyr)
print(class(df))
result <- df %>% count(UniqueCarrier)
print(result)
end_time <- Sys.time()
print(end_time - start_time)

[1] "data.frame"
   UniqueCarrier       n
1             AA 1220552
2             AS  273960
3             CO  756523
4             DL 1675173
5             HP  383652
6             NW  998094
7             TW  523951
8             UA 1374698
9             US 1350489
10            WN 1442908
Time difference of 0.2657518 secs
CPU times: user 5min 38s, sys: 18.8 s, total: 5min 57s
Wall time: 6min 12s


In [ ]:
## Now lets use the arrow format to excahnge the dataframes. Here is the github repo, 
# still in its baby state so use with caution. https://github.com/rpy2/rpy2-arrow
# Another way exchange data is by writing it to feather from python and then reading in R.
# try that way and check how much time it takes

In [18]:
%%time
%%memit
## read more on the datasets here  https://arrow.apache.org/docs/python/dataset.html
#Loading around 90 Million rows
dataset = ds.dataset("figshareairline/combined_data.csv", format="csv")
## this is of arrow table format
table = dataset.to_table()

peak memory: 15092.19 MiB, increment: 6074.00 MiB
CPU times: user 14.3 s, sys: 8.6 s, total: 22.9 s
Wall time: 23 s


In [19]:
%%time
%%memit
## Here we are loading the arrow dataframe that we have loaded previously
r_table = pyra.converter.py2rpy(table)

3164
rarrow.ChunkedArray: 0.03018498420715332
3164
rarrow.ChunkedArray: 0.03617095947265625
3164
rarrow.ChunkedArray: 0.030874013900756836
3164
rarrow.ChunkedArray: 0.03427600860595703
3164
rarrow.ChunkedArray: 0.03125905990600586
3164
rarrow.ChunkedArray: 0.03152108192443848
3164
rarrow.ChunkedArray: 0.03460121154785156
3164
rarrow.ChunkedArray: 0.03463602066040039
peak memory: 14650.72 MiB, increment: 4461.88 MiB
CPU times: user 8.51 s, sys: 2.1 s, total: 10.6 s
Wall time: 11.7 s


In [27]:
%%time
%%R -i r_table
start_time <- Sys.time()
print(class(r_table))
##add details on collect here
library(dplyr)
# Arrow only support some operations check this out https://arrow.apache.org/docs/r/articles/dataset.html
result <- r_table %>% collect() %>% count(UniqueCarrier)
print(class(r_table %>% collect()))
end_time <- Sys.time()
print(result)
print(end_time - start_time)

[1] "Table"       "ArrowObject" "R6"         
[1] "tbl_df"     "tbl"        "data.frame"
# A tibble: 24 x 2
   UniqueCarrier        n
   <chr>            <int>
 1 9E              521059
 2 AA             9672922
 3 AQ              154381
 4 AS             2162672
 5 B6              811341
 6 CO             4976761
 7 DH              693047
 8 DL            10435886
 9 EV             1697172
10 F9              336958
# … with 14 more rows
Time difference of 1.184519 mins
CPU times: user 20.9 s, sys: 20.4 s, total: 41.3 s
Wall time: 1min 11s


In [30]:
%%time
%%R
### her we are showing how much time it took to read a feather file what we wrote in python
library(arrow)
start_time <- Sys.time()
r_table <- arrow::read_feather("figshareairline/example.feather")
print(class(r_table))
library(dplyr)
result <- r_table %>% count(UniqueCarrier)
end_time <- Sys.time()
print(result)
print(end_time - start_time)

[1] "tbl_df"     "tbl"        "data.frame"
# A tibble: 24 x 2
   UniqueCarrier        n
   <chr>            <int>
 1 9E              521059
 2 AA             9672922
 3 AQ              154381
 4 AS             2162672
 5 B6              811341
 6 CO             4976761
 7 DH              693047
 8 DL            10435886
 9 EV             1697172
10 F9              336958
# … with 14 more rows
Time difference of 1.683477 mins
CPU times: user 21.2 s, sys: 36.4 s, total: 57.6 s
Wall time: 1min 41s
